Total Game Score Model - Model Tuning and Building - GBM

In [3]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')
from total_points_model.config import raw_data_file_path
from total_points_model.domain.preprocessing.data_preprocessor import DataPreprocessor
from total_points_model.domain.contracts.mappings import Mappings
from total_points_model.domain.contracts.modelling_data_contract import ModellingDataContract
from total_points_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner, XGBYearHyperparameterTuner
from total_points_model.domain.modelling.supermodel import SuperXGBRegressor

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Model Version

In [4]:
model_version = 2
model_name = 'xgb_total_points'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/total-points-score-model/afl-total-points-model/total_points_model/models/"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/total-points-score-model/afl-total-points-model/total_points_model/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/total-points-score-model/afl-total-points-model/total_points_model/preprocessors/"

Load Data

In [32]:
afl_data = pd.read_csv(raw_data_file_path)
afl_data = afl_data[afl_data['Year'] < 2023]

In [33]:
training_data = afl_data[afl_data['ModellingFilter']]
test_data = afl_data[~afl_data['ModellingFilter']]

In [34]:
response = ModellingDataContract.response

In [35]:
X_train, y_train = training_data.drop(columns = [response]), training_data[response]
X_test, y_test = test_data.drop(columns = [response]), test_data[response]

In [36]:
test_data.tail()

,Home_Team,Venue,Round_ID,Match_ID,Year,Away_Team,Q1_Score,Q2_Score,Q3_Score,Q4_Score,Margin,Total_Game_Score,Home_Win,City,Date,Attendance,Home_Coach_ID,Away_Coach_ID,Q5_Score,Temperature,Weather_Type,Match_Status,Weather_Description,Ground_Width,Ground_Length,Home_Ground,ModellingFilter,DateTime
3533,Melbourne,M.C.G.,2022F2,2022F2_Melbourne_BrisbaneLions,2022.0,Brisbane Lions,3.6.24 - 1.3.9,6.8.44 - 3.4.22,8.11.59 - 9.5.59,11.13.79 - 14.8.92,-13.0,171.0,0.0,Melbourne,2022-09-09,62162.0,Simon_Goodwin,Chris_Fagan,NaN,20.0,RAIN,CONCLUDED,Shower or two,141,160,Primary Home,False,2022-09-09 19:20:00
3534,Collingwood,M.C.G.,2022F2,2022F2_Collingwood_Fremantle,2022.0,Fremantle,4.0.24 - 0.1.1,6.6.42 - 2.2.14,8.12.60 - 4.4.28,11.13.79 - 9.5.59,20.0,138.0,1.0,Melbourne,2022-09-10,90612.0,Craig_McRae,Justin_Longmuir,NaN,15.0,RAIN,CONCLUDED,Showers,141,160,Primary Home,False,2022-09-10 18:55:00
3535,Geelong,M.C.G.,2022F3,2022F3_Geelong_BrisbaneLions,2022.0,Brisbane Lions,4.2.26 - 2.0.12,7.7.49 - 3.1.19,14.11.95 - 5.2.32,18.12.120 - 7.7.49,71.0,169.0,1.0,Melbourne,2022-09-16,77677.0,Chris_Scott,Chris_Fagan,NaN,17.0,RAIN,CONCLUDED,Rain at times,141,160,Neutral,False,2022-09-16 19:20:00
3536,Sydney,S.C.G.,2022F3,2022F3_Sydney_Collingwood,2022.0,Collingwood,6.3.39 - 3.0.18,11.7.73 - 7.1.43,13.10.88 - 10.5.65,14.11.95 - 14.10.94,1.0,189.0,1.0,Sydney,2022-09-17,45608.0,John_Longmire,Craig_McRae,NaN,23.0,MOSTLY_SUNNY,CONCLUDED,Partly cloudy,136,155,Primary Home,False,2022-09-17 16:15:00
3537,Geelong,M.C.G.,2022F4,2022F4_Geelong_Sydney,2022.0,Sydney,6.5.41 - 1.0.6,9.8.62 - 4.2.26,15.11.101 - 4.3.27,20.13.133 - 8.4.52,81.0,185.0,1.0,Melbourne,2022-09-24,100024.0,Chris_Scott,John_Longmire,NaN,15.0,RAIN,CONCLUDED,Morning shower or two,141,160,Neutral,False,2022-09-24 14:00:00


Preprocess Data

In [37]:
preprocessor = DataPreprocessor(mapping=Mappings.mappings)

In [38]:
preprocessor.fit(X_train)

DataPreprocessor(mapping={'Round': {'01': 1, '02': 2, '03': 3, '04': 4, '05': 5,
                                    '06': 6, '07': 7, '08': 8, '09': 9,
                                    '10': 10, '11': 11, '12': 12, '13': 13,
                                    '14': 14, '15': 15, '16': 16, '17': 17,
                                    '18': 18, '19': 19, '20': 20, '21': 21,
                                    '22': 22, '23': 23, '24': 24, 'F1': 25,
                                    'F2': 26, 'F3': 27, 'F4': 28, 'F5': 29},
                          'Weather_Type': {'CLEAR_NIGHT': 'Good',
                                           'MOSTLY_CLEAR': 'Good',
                                           'MOSTLY_SUNNY': 'Good',
                                           'OVERCAST': 'Good', 'RAIN': 'Bad',
                                           'ROOF_CLOSED': 'Good',
                                           'SUNNY': 'Good',
                                           'THUNDERSTORMS': 'Bad',
                                           'WINDY': 'Bad',
                                           'WINDY_RAIN': 'Bad'}})

In [39]:
X_train_preproc = preprocessor.transform(X_train)
X_test_preproc = preprocessor.transform(X_test)

In [40]:
X_train_preproc.head()

,Round,Year,Temperature,random5,Home_Total_Q4_Score_avg2,Home_Total_Q4_Goals_avg2,Home_Total_Q4_Behinds_avg2,Home_Total_Q4_Shots_avg2,Home_Total_Q4_Conversion_avg2,Home_Att_Q4_Score_avg2,Home_Att_Q4_Goals_avg2,Home_Att_Q4_Behinds_avg2,Home_Att_Q4_Shots_avg2,Home_Att_Q4_Conversion_avg2,Home_Def_Q4_Score_avg2,Home_Def_Q4_Goals_avg2,Home_Def_Q4_Behinds_avg2,Home_Def_Q4_Shots_avg2,Home_Def_Q4_Conversion_avg2,Away_Total_Q4_Score_avg2,Away_Total_Q4_Goals_avg2,Away_Total_Q4_Behinds_avg2,Away_Total_Q4_Shots_avg2,Away_Total_Q4_Conversion_avg2,Away_Att_Q4_Score_avg2,Away_Att_Q4_Goals_avg2,Away_Att_Q4_Behinds_avg2,Away_Att_Q4_Shots_avg2,Away_Att_Q4_Conversion_avg2,Away_Def_Q4_Score_avg2,Away_Def_Q4_Goals_avg2,Away_Def_Q4_Behinds_avg2,Away_Def_Q4_Shots_avg2,Away_Def_Q4_Conversion_avg2,Home_Team_Adelaide,Home_Team_Brisbane Lions,Home_Team_Carlton,Home_Team_Collingwood,Home_Team_Essendon,Home_Team_Fremantle,Home_Team_Geelong,Home_Team_Gold Coast,Home_Team_Greater Western Sydney,Home_Team_Hawthorn,Home_Team_Melbourne,Home_Team_North Melbourne,Home_Team_Port Adelaide,Home_Team_Richmond,Home_Team_St Kilda,Home_Team_Sydney,Home_Team_West Coast,Home_Team_Western Bulldogs,Away_Team_Adelaide,Away_Team_Brisbane Lions,Away_Team_Carlton,Away_Team_Collingwood,Away_Team_Essendon,Away_Team_Fremantle,Away_Team_Geelong,Away_Team_Gold Coast,Away_Team_Greater Western Sydney,Away_Team_Hawthorn,Away_Team_Melbourne,Away_Team_North Melbourne,Away_Team_Port Adelaide,Away_Team_Richmond,Away_Team_St Kilda,Away_Team_Sydney,Away_Team_West Coast,Away_Team_Western Bulldogs,Venue_Adelaide Oval,Venue_Bellerive Oval,Venue_Blacktown,Venue_Carrara,Venue_Cazalys Stadium,Venue_Docklands,Venue_Eureka Stadium,Venue_Football Park,Venue_Gabba,Venue_Jiangwan Stadium,Venue_Kardinia Park,Venue_M.C.G.,Venue_Manuka Oval,Venue_Marrara Oval,Venue_Perth Stadium,Venue_Princes Park,Venue_S.C.G.,Venue_Stadium Australia,Venue_Subiaco,Venue_Sydney Showground,Venue_Traeger Park,Venue_Wellington,Venue_York Park,City_Adelaide,City_Alice Springs,City_Ballarat,City_Brisbane,City_Cairns,City_Canberra,City_Darwin,City_Geelong,City_Gold Coast,City_Hobart,City_Launceston,City_Melbourne,City_Perth,City_Shanghai,City_Sydney,City_Wellington,Weather_Type_Bad,Weather_Type_Good
0,1,2005.0,18.0,4,181.188342,26.305829,23.35337,49.659199,0.528876,91.5,13.281603,11.810383,25.091985,0.526871,89.688342,13.024226,11.542987,24.567213,0.528248,181.774299,26.394976,23.404441,49.799418,0.52913,90.031853,13.076629,11.572079,24.648708,0.528787,91.742446,13.318347,11.832363,25.15071,0.52722,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,2005.0,18.0,4,181.188342,26.305829,23.35337,49.659199,0.528876,91.5,13.281603,11.810383,25.091985,0.526871,89.688342,13.024226,11.542987,24.567213,0.528248,181.774299,26.394976,23.404441,49.799418,0.52913,90.031853,13.076629,11.572079,24.648708,0.528787,91.742446,13.318347,11.832363,25.15071,0.52722,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,1,2005.0,18.0,1,181.188342,26.305829,23.35337,49.659199,0.528876,91.5,13.281603,11.810383,25.091985,0.526871,89.688342,13.024226,11.542987,24.567213,0.528248,181.774299,26.394976,23.404441,49.799418,0.52913,90.031853,13.076629,11.572079,24.648708,0.528787,91.742446,13.318347,11.832363,25.15071,0.52722,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,1,2005.0,18.0,5,181.188342,26.305829,23.35337,49.659199,0.528876,91.5,13.281603,11.810383,25.091985,0.526871,89.688342,13.024226,11.542987,24.567213,0.528248,181.774299,26.394976,23.404441,49.799418,0.52913,90.031853,13.076629,11.572079,24.648708,0.528787,91.742446,13.318347,11.832363,25.15071,0.52722,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,

In [41]:
X_train_preproc.shape, X_test_preproc.shape

((2755, 111), (783, 111))

Save preprocessor

In [42]:
joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")

['/Users/ciaran/Documents/Projects/AFL/total-points-score-model/afl-total-points-model/total_points_model/preprocessors/preprocessor_v2.joblib']

Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [50]:
xgb_tuner = XGBYearHyperparameterTuner(X_train_preproc, y_train)

In [53]:
xgb_tuner.tune_hyperparameters()

[I 2023-05-16 16:02:06,788] A new study created in memory with name: no-name-6c2723ce-0d88-42a8-9674-97bf7d516088


[0]	train-rmse:190.26098	valid-rmse:185.45936
[1]	train-rmse:188.17757	valid-rmse:183.39082
[2]	train-rmse:186.10594	valid-rmse:181.31186
[3]	train-rmse:184.05646	valid-rmse:179.30959
[4]	train-rmse:181.98250	valid-rmse:177.22802
[5]	train-rmse:179.90998	valid-rmse:175.17753
[6]	train-rmse:177.87786	valid-rmse:173.17632
[7]	train-rmse:175.87146	valid-rmse:171.19899
[8]	train-rmse:173.97030	valid-rmse:169.33927
[9]	train-rmse:172.19600	valid-rmse:167.60339
[10]	train-rmse:170.22278	valid-rmse:165.63150
[11]	train-rmse:168.32270	valid-rmse:163.72347
[12]	train-rmse:166.51562	valid-rmse:161.95613
[13]	train-rmse:164.63139	valid-rmse:160.09068
[14]	train-rmse:162.84001	valid-rmse:158.36713
[15]	train-rmse:161.10311	valid-rmse:156.70678
[16]	train-rmse:159.28414	valid-rmse:154.90591
[17]	train-rmse:157.50042	valid-rmse:153.17532
[18]	train-rmse:155.79340	valid-rmse:151.51970
[19]	train-rmse:154.14968	valid-rmse:149.92284
[20]	train-rmse:152.47147	valid-rmse:148.28015
[21]	train-rmse:150.782

[I 2023-05-16 16:02:39,993] Trial 0 finished with value: 27.060526150760573 and parameters: {'max_depth': 11, 'min_child_weight': 14, 'eta': 0.011670412994964805, 'gamma': 0.0004875969621371861, 'lambda': 0.40422336617102045, 'alpha': 0.11953186797842831, 'subsample': 0.24504196025459563, 'colsample_bytree': 0.4034082980540764}. Best is trial 0 with value: 27.060526150760573.


[0]	train-rmse:98.64745	valid-rmse:97.61502
[1]	train-rmse:53.63584	valid-rmse:55.82870
[2]	train-rmse:34.32784	valid-rmse:40.78275
[3]	train-rmse:25.60738	valid-rmse:37.93811
[4]	train-rmse:21.90845	valid-rmse:36.97763
[5]	train-rmse:20.18197	valid-rmse:37.77838
[6]	train-rmse:18.88529	valid-rmse:37.62718
[7]	train-rmse:17.43825	valid-rmse:39.42604
[8]	train-rmse:15.97929	valid-rmse:39.96395
[9]	train-rmse:15.36612	valid-rmse:41.05387
[10]	train-rmse:14.37987	valid-rmse:41.38023
[11]	train-rmse:13.06253	valid-rmse:41.93285
[12]	train-rmse:12.53866	valid-rmse:42.13453
[13]	train-rmse:11.91033	valid-rmse:42.66214
[14]	train-rmse:11.40984	valid-rmse:43.14509
[15]	train-rmse:10.62548	valid-rmse:43.30997
[16]	train-rmse:10.16511	valid-rmse:43.32170
[17]	train-rmse:10.03338	valid-rmse:43.10251
[18]	train-rmse:9.74950	valid-rmse:42.86411
[19]	train-rmse:9.19438	valid-rmse:42.99457
[20]	train-rmse:8.35628	valid-rmse:42.86281
[21]	train-rmse:8.41793	valid-rmse:42.87687
[22]	train-rmse:7.84656	

[I 2023-05-16 16:02:48,204] Trial 1 finished with value: 37.80265303520867 and parameters: {'max_depth': 8, 'min_child_weight': 5, 'eta': 0.5035020993042132, 'gamma': 0.0005517056133503287, 'lambda': 0.024347653523161785, 'alpha': 0.00043063714518588495, 'subsample': 0.5340949818877885, 'colsample_bytree': 0.382896935817461}. Best is trial 0 with value: 27.060526150760573.


...

[I 2023-05-16 16:05:56,153] Trial 12 finished with value: 26.663947184952583 and parameters: {'max_depth': 2, 'min_child_weight': 9, 'eta': 0.010083354133966086, 'gamma': 0.007321736409845373, 'lambda': 0.8295516395258175, 'alpha': 0.9996519231030649, 'subsample': 0.20306121131194263, 'colsample_bytree': 0.203047739546398}. Best is trial 12 with value: 26.663947184952583.


[0]	train-rmse:184.07214	valid-rmse:179.50245
[1]	train-rmse:176.37470	valid-rmse:171.96014
[2]	train-rmse:168.96858	valid-rmse:164.71187
[3]	train-rmse:162.12644	valid-rmse:158.07737
[4]	train-rmse:155.47375	valid-rmse:151.53784
[5]	train-rmse:149.12128	valid-rmse:145.40449
[6]	train-rmse:142.79576	valid-rmse:139.12679
[7]	train-rmse:136.76500	valid-rmse:133.17612
[8]	train-rmse:131.24778	valid-rmse:127.81349
[9]	train-rmse:126.18094	valid-rmse:122.66409
[10]	train-rmse:120.85576	valid-rmse:117.44516
[11]	train-rmse:116.12617	valid-rmse:112.67161
[12]	train-rmse:111.55877	valid-rmse:108.44411
[13]	train-rmse:107.15475	valid-rmse:103.98477
[14]	train-rmse:102.99906	valid-rmse:99.95137
[15]	train-rmse:98.99252	valid-rmse:96.21132
[16]	train-rmse:94.97110	valid-rmse:92.34423
[17]	train-rmse:91.08995	valid-rmse:88.53200
[18]	train-rmse:87.61153	valid-rmse:85.15184
[19]	train-rmse:84.48719	valid-rmse:82.05769
[20]	train-rmse:81.26247	valid-rmse:78.92764
[21]	train-rmse:78.21143	valid-rmse:

[W 2023-05-16 16:06:03,851] Trial 13 failed with parameters: {'max_depth': 2, 'min_child_weight': 9, 'eta': 0.04293969275926738, 'gamma': 0.006880065749788552, 'lambda': 0.11099747222743224, 'alpha': 0.7903068967956585, 'subsample': 0.31176833900356216, 'colsample_bytree': 0.2364296422720985} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/ciaran/opt/anaconda3/envs/afl-total-model-env/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/Users/ciaran/Documents/Projects/AFL/total-points-score-model/afl-total-points-model/notebooks/../total_points_model/domain/modelling/hyperparameter_tuning.py", line 170, in objective
    bst = xgb.train(param,
  File "/Users/ciaran/opt/anaconda3/envs/afl-total-model-env/lib/python3.9/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/Users/ciaran/opt/anaconda3/envs/afl-total-model-env/lib/pytho

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/ciaran/opt/anaconda3/envs/afl-total-model-env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/fm/_t15h8ls56q08qggzncfm73w0000gn/T/ipykernel_92737/2238644649.py", line 1, in <module>
    xgb_tuner.tune_hyperparameters()
  File "/Users/ciaran/Documents/Projects/AFL/total-points-score-model/afl-total-points-model/notebooks/../total_points_model/domain/modelling/hyperparameter_tuning.py", line 190, in tune_hyperparameters
    self.study.optimize(self.objective, n_trials=self.trials)
  File "/Users/ciaran/opt/anaconda3/envs/afl-total-model-env/lib/python3.9/site-packages/optuna/study/study.py", line 425, in optimize
    _optimize(
  File "/Users/ciaran/opt/anaconda3/envs/afl-total-model-env/lib/python3.9/site-packages/optuna/study/_optimize.py", line 66, in _optimize
    _optimize_sequential(
  File "/Users/ciaran/opt/anaconda3/en

In [54]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 2,
 'min_child_weight': 9,
 'eta': 0.010083354133966086,
 'gamma': 0.007321736409845373,
 'lambda': 0.8295516395258175,
 'alpha': 0.9996519231030649,
 'subsample': 0.20306121131194263,
 'colsample_bytree': 0.203047739546398}

Training Model - SuperXGBRegressor class for training and predictions

In [55]:
params['objective'] = 'reg:squarederror'
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1

In [56]:
super_xgb = SuperXGBRegressor(X_train = X_train_preproc, 
                              y_train = y_train, 
                              X_test = X_test_preproc, 
                              y_test = y_test, 
                              params = params)

In [57]:
super_xgb.fit()

[0]	validation_0-rmse:182.14162	validation_1-rmse:154.61528
[1]	validation_0-rmse:180.34751	validation_1-rmse:152.92164
[2]	validation_0-rmse:178.57900	validation_1-rmse:151.31210
[3]	validation_0-rmse:176.83252	validation_1-rmse:149.67281
[4]	validation_0-rmse:175.07310	validation_1-rmse:148.02836
[5]	validation_0-rmse:173.38689	validation_1-rmse:146.38961
[6]	validation_0-rmse:171.68628	validation_1-rmse:144.74291
[7]	validation_0-rmse:170.00732	validation_1-rmse:143.16925
[8]	validation_0-rmse:168.34991	validation_1-rmse:141.63236
[9]	validation_0-rmse:166.71366	validation_1-rmse:140.12933
[10]	validation_0-rmse:165.08847	validation_1-rmse:138.68202
[11]	validation_0-rmse:163.48334	validation_1-rmse:137.19531
[12]	validation_0-rmse:161.89088	validation_1-rmse:135.74921
[13]	validation_0-rmse:160.31800	validation_1-rmse:134.34178
[14]	validation_0-rmse:158.76697	validation_1-rmse:132.87894
[15]	validation_0-rmse:157.24421	validation_1-rmse:131.51508
[16]	validation_0-rmse:155.72367	v

In [58]:
super_xgb.xgb_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.203047739546398, early_stopping_rounds=50,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.007321736409845373, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.010083354133966086, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=9, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [59]:
test_preds = super_xgb.predict(X_test_preproc)

Export model

In [60]:
super_xgb.export_model(model_output_path + model_file_name + ".joblib")

Export data and predictions

In [67]:
train_export = training_data[['Match_ID', 'Home_Team', 'Away_Team', 'Round_ID', 'Total_Game_Score']]
train_export = pd.merge(train_export, X_train_preproc, how='left', left_index=True, right_index=True)
train_export['xgb_preds'] = super_xgb.predict(X_train_preproc)
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)

In [69]:
test_export = test_data[['Match_ID', 'Home_Team', 'Away_Team', 'Round_ID', 'Total_Game_Score']]
test_export = pd.merge(test_export, X_test_preproc, how='left', left_index=True, right_index=True)
test_export['xgb_preds'] = super_xgb.predict(X_test_preproc)
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)